In [1]:
import numpy as np
from util import *
DIR, loc, EEG_eval, EEGL_eval, EEGR_eval, EEGL_train, EEGR_train = A3Load()

In [2]:
np.random.seed(189)
rand_nums = np.random.permutation(6)

In [3]:
n_channel, n_time, n_sample = EEGR_train[0][0].shape
randIdx = np.random.permutation(n_sample)

In [4]:
k = 6
print(f'Our k is: {k}')

bin_size = int(n_sample / k)
print(f'Our bin size is: {bin_size}')

# In order to make our task easier, 
# we can reshape randIdx to have k rows and bin_size columns
# by using .reshape():
randIdx = randIdx.reshape(k, bin_size)
print(randIdx)
assert np.array_equal(randIdx,
                      np.array([[56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53],
                                [52, 16, 14, 21, 10, 64, 41, 47, 28,  5, 20, 42],
                                [13, 33,  7, 59, 18, 71, 49, 22, 68, 26, 61, 32],
                                [ 6, 46,  3, 48, 44,  8, 11, 15, 17,  1, 24,  0],
                                [30, 65, 12,  2, 27, 37, 55, 23, 69, 67, 39, 57],
                                [36, 29, 45, 60, 38, 40, 51, 62, 19, 25, 58, 35]]))

Our k is: 6
Our bin size is: 12
[[56 70 31 50 66 34 63  4 54  9 43 53]
 [52 16 14 21 10 64 41 47 28  5 20 42]
 [13 33  7 59 18 71 49 22 68 26 61 32]
 [ 6 46  3 48 44  8 11 15 17  1 24  0]
 [30 65 12  2 27 37 55 23 69 67 39 57]
 [36 29 45 60 38 40 51 62 19 25 58 35]]


In [5]:
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int)
valIdx = np.zeros((k, bin_size), dtype=int)

In [6]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

array([[56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53],
       [39, 59,  5,  7, 32, 36, 68,  9, 12, 60, 58, 10],
       [27, 35, 25, 63, 22, 49,  3, 20, 34, 33,  5, 28],
       [ 9, 36,  4, 18, 31, 54, 49, 30, 58, 42, 22, 34],
       [53, 34, 63, 10, 39, 61,  4, 51, 36, 69, 48, 23],
       [41,  6, 10, 23, 19, 29, 30, 59, 49, 39, 71, 51]])

In [7]:
# Depending on your solution, valIdx and trainIdx may not be np.arrays
# Let's guarantee they are
valIdx = np.array(valIdx)
trainIdx = np.array(trainIdx)

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

assert np.array_equal(valIdx[0,:], np.array([56, 70, 31, 50, 66, 34, 63,  4, 54,  9, 43, 53]))
assert np.array_equal(trainIdx[0,:], np.array([52, 16, 14, 21, 10, 64, 41, 47, 28,  5, 20, 42, 13, 33,  7, 59, 18,
                                               71, 49, 22, 68, 26, 61, 32,  6, 46,  3, 48, 44,  8, 11, 15, 17,  1,
                                               24,  0, 30, 65, 12,  2, 27, 37, 55, 23, 69, 67, 39, 57, 36, 29, 45,
                                               60, 38, 40, 51, 62, 19, 25, 58, 35]))

In [9]:
# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = EEGL_train.shape
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            L_train = EEGL_train[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_train = EEGR_train[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            L_val = EEGL_train[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_val = EEGR_train[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((L_train,R_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((L_val,R_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        accuracy[subject,fold] = sum(np.rint(band_score_val.mean(1))==y_val)/len(y_val)
    print('===============')
print(np.mean(accuracy, axis=1))

# Main Analysis Over

Processing Subject 1/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/9
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6